In [15]:
from nltk.corpus import wordnet


In [16]:
# From https://github.com/TrustAI/testRNN/blob/master/src/eda.py
def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word):
		for l in syn.lemmas():
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym)
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

In [17]:
# WordNet synonyms are VERY unreliable:

get_synonyms('water')

KeyboardInterrupt: 

In [ ]:
get_synonyms('want')


In [13]:
from gensim.models import KeyedVectors

word2vec_path = 'news_vectors.bin'
vectors = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

INFO:gensim.models.keyedvectors:loading projection weights from news_vectors.bin


KeyboardInterrupt: 

In [14]:
import torch
# from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import spacy
from dev_data import texts as dev_texts
import numpy as np

nlp_spacy = spacy.load('en_core_web_lg')

def get_sentences(text):
    """
    Use spaCy for sentence segmentation
    """
    sentences = []
    doc = nlp_spacy(text)
    for sent in doc.sents:
        sentences.append(str(sent))
    return np.array(sentences)

input_sentences = get_sentences(dev_texts[1][0])[:10]
#input_sentence = "They were there to enjoy us and they were there to pray for us."
#
# model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text, num_return_sequences, num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

print(dev_texts[1][0])
print(get_response(dev_texts[1][0], 40, 10))

# for input_sentence in input_sentences:
#     generated_sentence = get_response(input_sentence, 10, 10)
#     print('----------')
#     print('ORIGINAL:')
#     print(input_sentence)
#     print('PARAPHRASED:')
#     print(generated_sentence)



['The name of the constellation is Latin for lion, and to the ancient Greeks it was the Nemean Lion killed by the mythical Greek hero Heracles.']
